In [2]:
%matplotlib inline
import numpy as np
import torch
import matplotlib.pyplot as plt
torch.set_printoptions(edgeitems =2, linewidth =75)

In [3]:
t_c =[0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u =[35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [4]:
def model(t_u, w, b):
    return w*t_u+b 

def loss_fn(t_p, t_c):
    square_diffs = (t_p -t_c)**2
    return square_diffs.mean()

def dloss_fn(t_p, t_c):
    dsq_diffs = 2*(t_p - t_c) / t_p.size(0)
    return dsq_diffs

def dmodel_dw(t_u, w, b):
    return t_u

def dmodel_db(t_u, w, b):
    return 1.0
def grad_fn(t_u, t_c, t_p,w, b):
    dloss_dtp = dloss_fn(t_p, t_c)
    dloss_dw = dloss_dtp * dmodel_dw(t_u, w, b)
    dloss_db = dloss_dtp * dmodel_db(t_u, w, b)
    return torch.stack([dloss_dw.sum(), dloss_db.sum()])

In [6]:
params=torch.tensor([1.0, 0.0], requires_grad=True)
params.grad is None

True

In [8]:
loss = loss_fn(model(t_u, *params), t_c)
loss.backward()
params.grad
if params.grad is not None:
    print('reset gard...')
    params.grad.zero_()

reset gard...


In [10]:
def training_loop(n_epoches, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epoches+1):
        if params.grad is not None:
            params.grad.zero_()
        
        t_p = model(t_u, *params) 
        loss= loss_fn(t_p, t_c) 
        loss.backward()
        
        with torch.no_grad():
            params -= learning_rate * params.grad
        
        if epoch % 500 == 0:
            print('Epoch %d, Loss %f '%(epoch, float(loss)))
    return params

In [12]:
t_un = t_u*0.1
training_loop(n_epoches=5000, learning_rate= 1e-2,
params = torch.tensor([1.0, 0.0], requires_grad=True),
t_u = t_un, t_c = t_c)

Epoch 500, Loss 7.860115 
Epoch 1000, Loss 3.828538 
Epoch 1500, Loss 3.092191 
Epoch 2000, Loss 2.957698 
Epoch 2500, Loss 2.933134 
Epoch 3000, Loss 2.928648 
Epoch 3500, Loss 2.927830 
Epoch 4000, Loss 2.927679 
Epoch 4500, Loss 2.927652 
Epoch 5000, Loss 2.927647 


tensor([  5.3671, -17.3012], requires_grad=True)